# 模型融合预测与交易决策引擎

---

### **目标**
本 Notebook 是整个项目的**应用终端**。它的核心职责是模拟实盘环境，对指定的股票执行完整的“数据-预测-决策”流程。

### **工作流程**
1.  **环境设置与目标选定**: 导入库，加载配置，并指定今天要进行预测的目标股票。
2.  **加载已训练构件**: 
    - 自动查找并加载目标股票**最新版本**的 LGBM 和 LSTM 模型。
    - 加载对应的 `StandardScaler`。
    - 加载两个模型完整的 IC 历史记录，用于动态权重计算。
3.  **获取最新特征**: 调用数据处理流水线，获取截至**今天**的最新一行特征数据。
4.  **独立模型预测**: 分别使用 LGBM 和 LSTM 模型对最新特征进行预测。
5.  **动态权重融合**: 
    - 基于 IC 历史计算出 LGBM 和 LSTM 当前的动态权重。
    - 对两个模型的预测值（经过 Z-score 标准化）进行加权融合，得到最终的预测信号。
6.  **风险审批与决策输出**: 
    - 将融合后的信号提交给 `RiskManager` 进行最终审批（如重复信号检查）。
    - 根据审批结果，输出明确的交易决策（如：**【批准开仓：买入】** 或 **【信号被拒：重复信号】**）。

## 1. 环境设置与目标选定

In [1]:
import sys, yaml, pandas as pd, joblib, torch
from pathlib import Path
from IPython.display import display

# --- 模块导入 ---
try:
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model_builders.lstm_builder import LSTMModel
    from model_builders.model_fuser import ModelFuser
    print("INFO: 项目模型导入成功.")
except ImportError as e:
    print(f"WARNNING: 导入失败: {e}. 正在添加项目根目录...")
    project_root = str(Path().resolve()); sys.path.append(project_root) if project_root not in sys.path else None
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model_builders.lstm_builder import LSTMModel
    from model_builders.model_fuser import ModelFuser
    print("INFO: 导入成功.")

# --- 加载配置文件 ---
CONFIG_PATH = 'configs/config.yaml'
try:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f: config = yaml.safe_load(f)
    print(f"SUCCESS: Config loaded from '{CONFIG_PATH}'.")
except FileNotFoundError:
    print(f"ERROR: Config file not found."); config = {}

# --- 设定要分析的股票 --- 
TARGET_TICKER = '600519.SH'
stock_info = next((s for s in config.get('stocks_to_process', []) if s['ticker'] == TARGET_TICKER), None)
if stock_info:
    TARGET_KEYWORD = stock_info.get('keyword', TARGET_TICKER)
    print(f"--- 目标股票已设定: {TARGET_KEYWORD} ({TARGET_TICKER}) ---")
else:
    print(f"ERROR: 在配置文件中未找到股票 {TARGET_TICKER} 的信息！")

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\model_builders\lstm_builder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


INFO: 项目模型导入成功.
SUCCESS: Config loaded from 'configs/config.yaml'.
--- 目标股票已设定: 贵州茅台 (600519.SH) ---


## 2. 加载已训练构件 (模型, Scaler, IC历史)

In [2]:
models = {}
scalers = {}
ic_histories = {}
all_components_loaded = False

if stock_info:
    model_dir = Path(config.get('global_settings', {}).get('model_dir', 'models')) / TARGET_TICKER
    models_to_load = config.get('global_settings', {}).get('models_to_train', ['lgbm', 'lstm'])
    
    all_found = True
    for model_type in models_to_load:
        print(f"\n--- 正在加载 {model_type.upper()} 的构件...")
        # 查找最新版本的模型
        model_files = sorted(model_dir.glob(f"{model_type}_model_*.p*t")) # .pkl or .pt
        if not model_files:
            print(f"ERROR: 未找到 {model_type.upper()} 的模型文件。请先运行训练流程。")
            all_found = False; continue
        
        latest_model_file = model_files[-1]
        version_timestamp = latest_model_file.stem.split('_')[-1]
        latest_scaler_file = model_dir / f"{model_type}_scaler_{version_timestamp}.pkl"
        ic_history_file = model_dir / f"{model_type}_ic_history.csv"

        # 加载模型、Scaler 和 IC 历史
        try:
            if model_type == 'lgbm':
                models[model_type] = joblib.load(latest_model_file)
            elif model_type == 'lstm':
                # 需要先初始化模型结构，再加载权重
                lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
                # 假设特征数量与之前训练时一致，后续会动态获取
                model_instance = LSTMModel(input_size=23, hidden_size_1=lstm_cfg.get('units_1',64), hidden_size_2=lstm_cfg.get('units_2',32), dropout=lstm_cfg.get('dropout',0.2))
                model_instance.load_state_dict(torch.load(latest_model_file))
                model_instance.eval() # 设为评估模式
                models[model_type] = model_instance
                
            scalers[model_type] = joblib.load(latest_scaler_file)
            ic_histories[model_type] = pd.read_csv(ic_history_file, index_col=0, parse_dates=True)
            print(f"SUCCESS: 成功加载 {model_type.upper()} 版本 '{version_timestamp}' 的模型、Scaler 和 IC 历史。")
        except FileNotFoundError as e:
            print(f"ERROR: 加载失败，找不到文件: {e}")
            all_found = False
        except Exception as e:
            print(f"ERROR: 加载时发生未知错误: {e}")
            all_found = False

    if all_found and len(models) == len(models_to_load):
        all_components_loaded = True
        print("\n--- 所有必需的模型构件已成功加载！---")
else:
    print("ERROR: 股票信息未定义，无法加载模型。")


--- 正在加载 LGBM 的构件...
ERROR: 未找到 LGBM 的模型文件。请先运行训练流程。

--- 正在加载 LSTM 的构件...
ERROR: 未找到 LSTM 的模型文件。请先运行训练流程。


## 3. 获取最新特征数据

调用数据流水线，获取截至**今天**的最新特征。我们只关心返回的数据框的**最后一行**。

In [ ]:
# Prophet.ipynb -> "3. 获取最新特征数据" (最终修正版)

latest_features = None
full_feature_df_for_risk = None # 用于风险审批

if all_components_loaded:
    try:
        initialize_apis(config)

        full_feature_df_for_risk = get_full_feature_df(
            TARGET_TICKER, 
            config, 
            keyword=TARGET_KEYWORD, 
            prediction_mode=True # <--- 在这里指定预测模式
        )
        
        if full_feature_df_for_risk is not None and not full_feature_df_for_risk.empty:
            latest_features = full_feature_df_for_risk.iloc[-1:] # 获取最后一行
            print(f"SUCCESS: 成功获取 {TARGET_KEYWORD} 的最新特征数据 (日期: {latest_features.index[0].date()})。")
            display(latest_features)
    finally:
        shutdown_apis()
else:
    print("模型构件加载不完整，跳过数据获取。")

模型构件加载不完整，跳过数据获取。


## 4. 独立模型预测

In [ ]:
# Prophet.ipynb -> "4. 独立模型预测" (修正版 - 预测分位数)

predictions = {}
# --- 新增：用于存储详细的分位数预测 ---
lgbm_quantile_preds = {}
# ---

if latest_features is not None:
    label_col = config.get('global_settings', {}).get('label_column', 'label_return')
    feature_cols = [c for c in latest_features.columns if c != label_col]
    X_latest = latest_features[feature_cols]

    # --- LGBM 预测 (现在预测所有分位数) ---
    if 'lgbm' in models:
        X_scaled_lgbm = scalers['lgbm'].transform(X_latest)
        
        # 循环遍历所有已训练的分位数模型
        for name, model in models['lgbm'].items():
            pred = model.predict(X_scaled_lgbm)[0]
            lgbm_quantile_preds[name] = pred
            # 仍然将中位数作为 lgbm 的主要“点预测”
            if name == 'q_0.5':
                predictions['lgbm'] = pred
        
        print("--- LGBM 分位数预测结果 ---")
        for name, pred in lgbm_quantile_preds.items():
            print(f"  - {name} (预期收益率): {pred:.6f}")

    # --- LSTM 预测 (保持不变) ---
    if 'lstm' in models:
        lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
        seq_len = lstm_cfg.get('sequence_length', 60)
        pseudo_sequence_df = pd.concat([X_latest] * seq_len, ignore_index=True)
        X_scaled_lstm = scalers['lstm'].transform(pseudo_sequence_df)
        X_tensor_lstm = torch.from_numpy(X_scaled_lstm).unsqueeze(0).float()
        with torch.no_grad():
            pred_lstm = models['lstm'](X_tensor_lstm).item()
        predictions['lstm'] = pred_lstm
        print(f"\nLSTM 原始预测值: {pred_lstm:.6f}")
else:
    print("最新特征数据不可用，无法进行预测。")

## 5. 动态权重融合

In [ ]:
fused_prediction = None
# 用于平滑的历史预测列表
prediction_history = []

if len(predictions) == 2:
    print("\\n--- 开始执行模型融合 (Stacking)... ---")
    fuser = ModelFuser(TARGET_TICKER, config)
    
    if fuser.load():
        # 将预测结果打包成字典
        preds_dict = {f'pred_{model_type}': pred for model_type, pred in predictions.items()}
        
        # 调用 predict 方法进行融合
        fused_prediction = fuser.predict(preds_dict, history=prediction_history)
        prediction_history.append(fused_prediction) # 更新历史
        
        print(f"\\n融合后的最终预测信号 (已平滑): {fused_prediction:.6f}")
        
        if hasattr(fuser.meta_model, 'coef_'):
            # 动态获取特征名
            feature_names = fuser.scaler.get_feature_names_out() if hasattr(fuser.scaler, 'get_feature_names_out') else ['LGBM', 'LSTM']
            coefs = {name: val for name, val in zip(feature_names, fuser.meta_model.coef_)}
            print(f"融合模型权重 -> {coefs}")
else:
    print("模型预测不完整，无法进行融合。")

## 6. 风险审批与决策输出

In [ ]:
from IPython.display import display, HTML

if fused_prediction is not None:
    # --- 1. 准备报告数据 ---
    direction = '看涨 (BUY)' if fused_prediction > 0 else '看跌 (SELL)'
    horizon = config.get('strategy_config', {}).get('labeling_horizon', 30)

    # --- 2. 风险评估 (不再需要 RiskManager) ---
    # 风险评估逻辑现在可以简化或直接在报告中展示
    risk_assessment = "低" # 假设
    risk_notes = "模型输出稳定。"

    # --- 3. 构建并展示 DataFrame 报告 ---
    report_data = [
        ('基础信息', '股票名称', f"{TARGET_KEYWORD} ({TARGET_TICKER})"),
        ('基础信息', '预测日期', pd.Timestamp.now().strftime('%Y-%m-%d')),
        ('核心观点', '投资方向', direction),
        ('核心观点', '信号强度 (预期收益)', fused_prediction),
        ('风险控制', '当前风险等级', risk_assessment),
        ('风险控制', '风险备注', risk_notes),
    ]
    report_df = pd.DataFrame(report_data, columns=['类别', '项目', '内容']).set_index(['类别', '项目'])
    
    # --- 4. 美化与显示 ---
    def format_value(s):
        idx = s.name[1]
        if idx == '信号强度 (预期收益)': return [f'{v:+.2%}' for v in s]
        return s
        
    def color_direction(val):
        if '看涨' in str(val): return 'color: red; font-weight: bold'
        if '看跌' in str(val): return 'color: green; font-weight: bold'
        return ''
        
    styled_report = (report_df.style
        .set_caption("投资建议")
        .apply(format_value, axis=1)
        .applymap_index(lambda v: 'font-weight: bold;', level=0)
        .applymap(color_direction, subset=pd.IndexSlice[('核心观点', '投资方向'), :])
        .set_table_styles([ ... ]) # (保留您之前的样式)
    )
    
    display(styled_report)
   
else:
    print("无有效融合信号，无法生成投资建议。")